# Mid-Term Project: Inventory System (110 points)
### `Objective:`
Develop a Python-based inventory management system that allows users to manage inventory, process purchases, track customer information, generate reports, and identify top customers. The project includes core functionalities and bonus features for extra credit.

### Core Requirements (80 points total):
* Inventory `Class` (5 points): Implement a class named Inventory to manage items in stock.
* Add Items (10 points): Implement a method `add_item` to add items to the inventory with the following attributes:
    - `Item ID`: Manually assigned unique identifier for each item.
    - `Item Name`: Name of the item.
    - `Item Brand`: Brand of the item, which helps to uniquely distinguish items with the same price and attributes.
    - `Price`: Cost of the item.
    - `Initial Quantity`: Total quantity available in stock.
    - `Update Date`: Date of last modification.
* Customer Management (10 points): Implement a method `assign_customer_id` to generate or retrieve a Customer ID based on the customer's email:
    - If a customer is purchasing for the first time, assign a new Customer ID.
    - If the customer already exists, retrieve their Customer ID from the historical data (e.g., CSV) or from the system

* Update Item Price (5 points): Implement a method `update_price` to update an item’s price by its ID.
* Purchase Items (15 points): Implement a method `purchase_item` to:
    - Update the item quantity after a purchase based on the Item ID.
    - Accept customer name and email for each purchase.
    - Automatically assign or retrieve the customer’s ID.
    - Check if the inventory quantity is sufficient and raise an error if not.
    - Track purchase history with automatic date capture.
* Check Inventory (10 points): Implement a method `get_item` to retrieve an item’s details and check if stock is running low. Low stock is defined as a quantity below a threshold you determine (e.g., fewer than 3 items).

* Identify Top Customers from CSV (15 points): Implement a method to identify customers who have spent more than 5000 dollars based on the purchase history.

* Provided Test Cases (10 points): Ensure the given test cases are successfully passed.


### Bonus Features (20 points total):
- Update CSV Files (10 points): After each purchase, update a CSV file to log the purchase history and inventory status.
- Install as a Package (10 points): Provide a setup.py file to enable the system to be installed as a Python package.
- You can also implement additional functions to make the classes more robust. For example, ensure consistency in letter cases. By default, the item type and item brand should be in upper case, and customer initials should be capitalized. However, if sales representatives accidentally type in lower case, you should create a function to automatically convert them to upper case or lower case based on system requirements. **Each additional function will be awarded up to 5 points, with a total of 15 points applied.** 

-------------------------------------------
### Project Timeline and Submission Guidelines:
Today’s Date: October 11th, 2024

Pseudo Code/Plan Submission Deadline: `October 22nd, 2024`

By this date, you are expected to submit pseudo code or descriptive ideas of how you plan to approach the project. This should outline your class structures, functions, and how you intend to meet the project requirements. (10 points)

### ChatGPT Usage Rules:
You are allowed to use ChatGPT for specific questions, such as:
- How to extract the current date.
- How to write a setup.py file.
- **You must explicitly indicate where you have used ChatGPT.**
- ### `Not allowed`: **Asking ChatGPT to directly write classes, as this would violate the honor code.**


### Final Project Submission:

Final project submission date: You should roughly have 3-4 weeks to finish it, precise date will be given after October 28th, 2024

Your final project should include:
- Completed code and documentation.
- Test cases.
- A clear indication of ChatGPT usage (if any).

In [1]:
import csv
import os
from collections import defaultdict
from datetime import datetime

# class to manage items in stock
class Inventory:
    def __init__(self, low_stock_min=3):
        self.items = {}
        self.customers = Customer_Management() # customer management instance / class
        self.purchases = Purchasing() # purchase instance / class
        self.low_stock_min = low_stock_min # getting the threshold for low stocks

    # error: cannot access local variable 'purchase_date' where it is not associated with a value
    # asked chatgpt for help with date variable in code
    def add_item(self, item_id, name, brand, price, quantity, date):
        if date is None:
            date = datetime.now().strftime("%Y-%m-%d") # current date
        self.items[item_id] = {"itemName": name, "itemBrand": brand,
                               "price": price, "quantity": quantity,
                               "date": date}

    def update_price(self, item_id, new_price):
        # updates price of existing item
        # returns success or none if item is non-existent
        if item_id in self.items:
            self.items[item_id]["price"] = new_price
            return "Price updated."
        return None

    def purchase_item(self, item_id, quantity, customer_name, customer_email):
        # aids purchase, updates quantity, and records the purchase
        if item_id not in self.items:
            raise ValueError("Item not found.")
        item = self.items[item_id]
        if item["quantity"] < quantity:
            # error if item low stocked or not found
            raise ValueError("Low stock.")
        # updates quantity stock
        item["quantity"] -= quantity

        # saves the purchase to the customer
        customer_id = self.customers.assign_customer_id(customer_name, customer_email)
        purchase_date = datetime.now().strftime("%Y-%m-%d")
        self.purchases.save_purchase(item_id, item["itemName"], customer_name, customer_email,
                                     item["price"] * quantity, purchase_date)

    # TypeError: string indices must be integers, not 'str' - asked perplexity.ai what to do
    # told me to create a copy of the item dict
    def get_item(self, item_id):
        # gets item details with a low stock warning
        item = self.items.get(item_id)
        if not item:
            return None # item has not been found
        item_copy = item.copy()
        if item["quantity"] < self.low_stock_min:
            item_copy["low_stock"] = True
            item_copy["low_stock_message"] = f"LOW STOCKED, {item['quantity']} remaining."
        else:
            item_copy["low_stock"] = False
        return item_copy


# bonus in attempt to make it more robust with separate class instead of 1 inventory class
# manages customers info with their IDs with class
class Customer_Management:
    def __init__(self):
        self.customers = {} # customer emails maping
        self.customer_count = 0 

    # assigns id to a customer
    # if email already in system, pulls it up
    def assign_customer_id(self, name, customer_email):
        if customer_email not in self.customers:
            self.customer_count += 1
            self.customers[customer_email] = {"id": self.customer_count, "name": name}
        return self.customers[customer_email]["id"]

    # gets customer name 
    def get_customer_name(self, email):
        return self.customers.get(email, {}).get("name")

    # chatgpt help here with reading in csv files
    def get_top_customers(self, file_path="store_data/purchase_history.csv", min_spent=5000):

        customer_spending = {}
        with open(file_path, newline="") as file:
            reader = csv.DictReader(file)
            for row in reader:
                email = row["customer_email"]
                amount = float(row["amount"])
            
                # if customer is not in the dictionary - add them
                if email not in customer_spending:
                    customer_spending[email] = 0
            
                # add purchase amount to the customer's total spending
                customer_spending[email] += amount

        # list of top customers who spent more than $5000
        top_customers = []
        for email, spent in customer_spending.items():
            if spent >= min_spent:
                top_customers.append({
                    "email": email,
                    "name": self.get_customer_name(email),
                    "total_spent": spent
                })
        return top_customers
        
# bonus in attempt to make it more robust with separate class
# asked perplexity.ai about properly connecting csv to package and classes
# also got help on how to read them, since I had file_path issues
class Purchasing:
    def __init__(self, file_path="store_data/purchase_history.csv"):
        self.file_path = file_path
        self._ensure_file_exists()

    # chatgpt help with making sure the csv file exists correctly
    def _ensure_file_exists(self):
        if not os.path.exists(self.file_path):
            with open(self.file_path, "w", newline = "") as file:
                writer = csv.writer(file)
                writer.writerow(["item_id", "item_name", "customer_name", 
                                 "customer_email", "amount", "purchase_date"])

    # makes sure to save purchases in csv
    def save_purchase(self, item_id, item_name, customer_name, customer_email, amount, purchase_date):        
        with open(self.file_path, "a", newline = "") as file:
            writer = csv.writer(file)
            writer.writerow([item_id, item_name, customer_name, customer_email, amount, purchase_date])


In [2]:
import sys
import os

sys.path.append("/Users/psorm/inventory_system")

from Inventory.inventory import Inventory

def test_add_item():
    inventory = Inventory()
    
    ### arguments: item ID, item type, item brand, item price, item quantity, item add date
    inventory.add_item("LAP001", "LAPTOP", "BRANDX", 1200, 5, "2024-10-11")  # Manually assigned Item ID
    item = inventory.get_item("LAP001")
    assert item['quantity'] == 5
    print("test_add_item passed:", item)

def test_update_price():
    inventory = Inventory()
    inventory.add_item("LAP001", "LAPTOP", "BRANDX", 1200, 5, "2024-10-11")
    
    ### arguments: item ID, item price
    inventory.update_price("LAP001", 1300)
    item = inventory.get_item("LAP001")
    assert item['price'] == 1300
    print("test_update_price passed:", item)

def test_purchase_item():
    inventory = Inventory()
    inventory.add_item("LAP001", "LAPTOP", "BRANDX", 1200, 5, "2024-10-11")
    ### item ID, purchase quantity, customer name, customer ID
    inventory.purchase_item("LAP001", 2, "John Doe", "john@example.com")  
    item = inventory.get_item("LAP001")
    assert item['quantity'] == 3
    print("test_purchase_item passed:", item)

def test_low_stock():
    inventory = Inventory()
    inventory.add_item("LAP001", "LAPTOP", "BRANDX", 1200, 2, "2024-10-11")
    item = inventory.get_item("LAP001")
    assert item['quantity'] == 2
    assert item['quantity'] < 3
    print("test_low_stock passed:", item)


# Execute all test functions and print outputs
if __name__ == "__main__":
    test_add_item()
    test_update_price()
    test_purchase_item()
    test_low_stock()

# chatgpt adjustments for easy code output printing

test_add_item passed: {'itemName': 'LAPTOP', 'itemBrand': 'BRANDX', 'price': 1200, 'quantity': 5, 'date': '2024-10-11', 'low_stock': False}
test_update_price passed: {'itemName': 'LAPTOP', 'itemBrand': 'BRANDX', 'price': 1300, 'quantity': 5, 'date': '2024-10-11', 'low_stock': False}
test_purchase_item passed: {'itemName': 'LAPTOP', 'itemBrand': 'BRANDX', 'price': 1200, 'quantity': 3, 'date': '2024-10-11', 'low_stock': False}
test_low_stock passed: {'itemName': 'LAPTOP', 'itemBrand': 'BRANDX', 'price': 1200, 'quantity': 2, 'date': '2024-10-11', 'low_stock': True, 'low_stock_message': 'LOW STOCKED, 2 remaining.'}


In [ ]:
def test_top_customers():
    inventory = Inventory()
    inventory.add_item("LAP001", "LAPTOP", "BRANDX", 1200, 10, "2024-10-11")
    inventory.purchase_item("LAP001", 5, "John Doe", "john@example.com")  
    inventory.purchase_item("LAP001", 1, "Jane Doe", "jane@example.com")
    top_customers = inventory.customers.get_top_customers("store_data/purchase_history.csv", min_spent=5000)
    assert len(top_customers) == 1
    assert top_customers[0]["name"] == "John Doe"
    assert top_customers[0]["email"] == "john@example.com"
    assert top_customers[0]["total_spent"] == 6000
    print("test_top_customers passed:", top_customers)

if __name__ == "__main__":
    test_top_customers()